### TODO
1. ~~Code up VAE~~
2. ~~Code up M (RNN)~~
3. ~~Code up Train Loop for M~~
3. ~~Code up C (Controller)~~
4. ~~Set up Car Racing data for C~~
5. set up car racing for VAE 
6. add in nice way to save weights and rollouts in same dir for VAE
6. set up VAE data with MDN_RNN
7. add temperature to MDN-RNN
8. add model saving to MDN-RNN
9. Get C working with VAE, MDN_RNN weights

In [1]:
import gym
import matplotlib.pyplot as plt

import torch
import numpy as np
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
from torch.nn import init
from torch import optim
import time
from copy import deepcopy
from tqdm import tqdm


import os
# Imports specifically so we can render outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display
import sys

# if "ipykernel_launcher" in sys.argv[0]:
#     sys.argv = [""]
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--weights_file",type=str,default="None")
parser.add_argument("--lr",type=float, default=0.0001)
parser.add_argument("--ctl_type",type=str, default="lstm")
parser.add_argument("--opt",type=str, default="adam")
parser.add_argument("--iters",type=int, default=100000)
#args = parser.parse_args()


_StoreAction(option_strings=['--iters'], dest='iters', nargs=None, const=None, default=100000, type=<class 'int'>, choices=None, help=None, metavar=None)

In [2]:
def show_envs(key=""):
    aenv = envs.registry.all()
    all_envs = str(aenv).replace("EnvSpec","").replace("dict_values", "").replace("(","").replace(")","").replace("[","").replace("]","").split(", ")
    for env in all_envs:
        if key == "":
            print(env)
        else:
            if key in env:
                print(env)

In [3]:

def display_frames_as_gif(frames,save=False):
    """
    Displays a list of frames as a gif, with controls
    """
    #plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    if save:
        anim.save('./hey.gif', writer='imagemagick', fps=60)

    display(display_animation(anim, default_mode='loop'))

In [4]:
class C(nn.Module):
    def __init__(self,env="CarRacing"):
        super(C,self).__init__()
        if env == "CarRacing":
            self.nz = 32
            self.nh = 256
            self.action_len = 3 
        self.fc = nn.Linear(in_features=self.nz+self.nh,out_features=self.action_len)
    
    def posprocess_output(self,raw_output):
        raw_steer, raw_gas, raw_brake = raw_output[0],raw_output[1],raw_output[2]

        steer = F.tanh(raw_steer) # between -1 and 1

        gas = F.softplus(raw_gas) # between 0 and 1

        brake = F.softplus(raw_brake) # between 0 and 1
        return steer,gas,brake
        
    
    def forward(self,z,h):
        z = z.squeeze()
        h = h.squeeze()
        zh = torch.cat((z,h),dim=-1)
        raw_output = self.fc(zh)
        steer, gas, brake = self.posprocess_output(raw_output)
        action = torch.cat((steer,gas,brake))
        return action.data
        

In [5]:
batch_size=1
c = C().cuda()

seq_len=1

env_name = "CarRacing-v0"
env = gym.make(env_name)
frames = []
env.reset()
s = env.render("rgb_array")
for i in range(100):
    z = Variable(torch.Tensor(seq_len,batch_size,c.nz).normal_()).cuda()
    h = Variable(torch.Tensor(seq_len,1,batch_size,c.nh).normal_()).cuda()
    a = c(z,h)
    _,r,d,_ = env.step(a)
    s = env.render("rgb_array")
    frames.append(s)
#display_frames_as_gif(frames)   
    

AssertionError: Torch not compiled with CUDA enabled